In [ ]:
import rasterio as rio
import os

In [ ]:
def get_subfolders(folder_path):
    return [os.path.join(folder_path, folder_name) for folder_name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, folder_name))]

def read_band(file_path, band_number):
    return rio.open(file_path, dtype='float32').read(band_number, masked=False)

def merge_bands(bands):
    return [bands[2], bands[1], bands[0], bands[3]]

def save_stack(out_img, out_meta, bands):
    with rio.open(out_img, 'w', **out_meta) as dest:
        for band_nr, src in enumerate(bands, start=1):
            dest.write(src, band_nr)

def process_subfolder(folder_path, output_folder):
    file_paths = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path)]

    bands = [read_band(file_paths[i], 1) for i in range(4)]

    file_name = os.path.basename(folder_path)
    complete_path_to_save = os.path.join(output_folder, file_name + '.tif')

    out_meta = rio.open(file_paths[2], dtype='float32').meta.copy()
    out_meta.update({"count": 4, "nodata": -9999})

    save_stack(complete_path_to_save, out_meta, merge_bands(bands))

if __name__ == "__main__":
    folder_path = 'path_to_folder_with_subfolders'  # Specify the path to the folder with subfolders
    output_folder = 'path_to_output_folder'  # Specify the path to the output folder

    subfolders = get_subfolders(folder_path)

    for subfolder in subfolders:
        process_subfolder(subfolder, output_folder)